# 快速开始函数式（Functional）模型

我们起初将Functional一词译作泛型，想要表达该类模型能够表达任意张量映射的含义，但表达的不是很精确，在Keras 2里面我们将这个词改译作“函数式”，对函数式编程有所了解的同学应该能够快速get到该类模型想要表达的含义。函数式模型称作Functional，但它的类名是Model，因此我们有时候也用Model来代表函数式模型。

Keras函数式模型接口是用户定义多输出模型、非循环有向模型或具有共享层的模型等复杂模型的途径。一句话，只要你的模型不是类似VGG一样一条路走到黑的模型，或者你的模型需要多于一个的输出，那么你总应该选择函数式模型。函数式模型是最广泛的一类模型，序贯模型（Sequential）只是它的一种特殊情况。

这部分的文档假设你已经对Sequential模型比较熟悉。

让我们从简单一点的模型开始。

## 第一个模型：全连接网络

Sequential当然是实现全连接网络的最好方式，但我们从简单的全连接网络开始，有助于我们学习这部分的内容。在开始前，有几个概念需要澄清：
- 层对象接收张量为参数，返回一个张量。
- 输入是张量，输出也是张量的一个框架就是一个模型，通过Model定义。
- 这样的模型可以被像Keras的Sequential一样被训练。

In [5]:
from __future__ import print_function

import keras
from keras.layers import Input, Dense
from keras.models import Model

# This returns a tensor
inputs = Input(shape=(784, ))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Generate dummy data
import numpy as np
x_train = np.random.random((1000, 784))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10)
x_test = np.random.random((100, 784))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)

model.fit(x_train, y_train, batch_size=128, epochs=20) # starts training

score = model.evaluate(x_test, y_test, batch_size=128)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/20
1000/1000 [==============================] - 0s 117us/step - loss: 2.3978 - acc: 0.0880
Epoch 2/20
1000/1000 [==============================] - 0s 21us/step - loss: 2.3072 - acc: 0.1170
Epoch 3/20
1000/1000 [==============================] - 0s 21us/step - loss: 2.2956 - acc: 0.1290
Epoch 4/20
1000/1000 [==============================] - 0s 22us/step - loss: 2.2860 - acc: 0.1380
Epoch 5/20
1000/1000 [==============================] - 0s 24us/step - loss: 2.2715 - acc: 0.1330
Epoch 6/20
1000/1000 [==============================] - 0s 22us/step - loss: 2.2727 - acc: 0.1340
Epoch 7/20
1000/1000 [==============================] - 0s 23us/step - loss: 2.2513 - acc: 0.1750
Epoch 8/20
1000/1000 [==============================] - 0s 23us/step - loss: 2.2261 - acc: 0.2130
Epoch 9/20
1000/1000 [==============================] - 0s 24us/step - loss: 2.2290 - acc: 0.1710
Epoch 10/20
1000/1000 [==============================] - 0s 22us/step - loss: 2.2135 - acc: 0.2170
Epoch 11/20
1000/1

## 所有的模型都是可调用的，就像层一样

利用函数式模型的接口，我们可以很容易的重用已经训练好的模型；你可以把模型当作一个层一样，通过提供一个tensor来调用它。注意当你调用一个模型时，你不仅仅重用了它的结构，也重用了它的权重。
``` python
x = Input(shape=(784, ))
# This works, and returns the 10-way softmax we defined above.
y = model(x)
```

这种方式可以允许你快速的创建能处理序列信号的模型，你可以很快将一个图像分类的模型变为一个对视频分类的模型，只需要一行代码：
``` python
from keras.layers import TimeDistributed

# Input tensor for sequences of 20 timesteps,
# each containings a 784-dimensional vector
input_sequences = Input(shape=(20, 784))

# This applies our previous model to every timestep in the input sequences.
# the output of the previous model was a 10-way softmax.
# so the output of the layer below will be a sequence of 20 vectors of size 10.
processed_sequences = TimeDistributed(model)(input_sequences)
```

## 多输入和多输出模型

